In this week, you will continue working on your capstone project. Please remember by the end of this week, you will need to submit the following:

A full report consisting of all of the following components (15 marks):
Introduction where you discuss the business problem and who would be interested in this project.
Data where you describe the data that will be used to solve the problem and the source of the data.
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
Results section where you discuss the results.
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
Conclusion section where you conclude the report.
2. A link to your Notebook on your Github repository pushed showing your code. (15 marks)

3. Your choice of a presentation or blogpost. (10 marks)

The COVID-19 pandemic, aside from its immediate effects, has led to uncertainty over the healthcare of vulnerable people, especially the elderly and those with pre-existing conditions. Many of these people now face a dilemma when it comes to accessing their healthcare needs since they may feel that visiting a hospital or clinic could expose them to avoidable risks; this is especially the case where existing services are stretched and may be experienced even in countries with socialised healthcare provision.
There may be a gap in the market for both public and private pharmacies to do more in local communities to ensure provision of basic medicines and equipment and for all people, but especially at-risk groups.

I will look at data across Sweden by municipality. There are 290 municipalities and the relevant data will be:
- Population per municipality, from Sweden's national statistics (http://www.statistikdatabasen.scb.se/)
- Area of the municipality, to create a view of population density; it is the total population that is important but population density may give a good measure of average travel distance to reach services
- Average age per municipality
- Pharmacies across Sweden by municipality from Foursquare

The main metrics to consider will be:
1. pharmacies per capita by municipality
2. pharmacies per capita by average age
3. pharmacy 'density' versus population density

It will be possible to present recommendations to private companies to show in which Swedish municipalities there are opportunities for new private pharmacies.

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

In [ ]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_municipalities_of_Sweden")
df=data[1]
df.head()


In [ ]:
df.drop(['Inland water area (km2)', 'Seawater area (km2)', 'Total area (km2)', 'Municipality', 'Nr'], axis=1, index=None, columns=None, level=None, inplace=True, errors='ignore')
df.head()

In [ ]:
!wget -q -O 'SE_age_municipality_edited.csv' https://raw.githubusercontent.com/neilrmackin/Coursera-_Capstone/master/SE_age_municipality_edited.csv
df_ages = pd.read_csv('SE_age_municipality_edited.csv')
df_ages.head()

In [ ]:
df_merge_col = pd.merge(df, df_ages, on='Code')
df_merge_col.head()


In [127]:
# Import data from Foursquare

CLIENT_ID = 'V04KOCNRBTU1RTELZ24JKXDJLSRN4QXCS30BRDK0R2LJMGMH' #  Foursquare ID
CLIENT_SECRET = 'UCCZMNLAWTT2KDJORWDKGNN1UGZ5QWF1OIPOEFWGU3RCXHRH' # Foursquare Secret
VERSION = '20180604'
LIMIT = 1000

In [140]:
address = 'Nödinge-Nol'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

57.9068771 12.0601607


In [146]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 

print("success")

success


In [ ]:
# add latitude and longitude of municipality seats to table df_merge_col
   
# declare an empty list to store 
# latitude and longitude of values  
# of city column 
longitude = [] 
latitude = [] 

# function to find the coordinate 
# of a given city  
def findGeocode(city): 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="foursquare_agent") 
          
        return geolocator.geocode(city) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(city) 

# each value from city column 
# will be fetched and sent to 
# function find_geocode    
for i in (df_merge_col["Seat"]): 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan)
        
        # now add this column to dataframe 
df_merge_col["Longitude"] = longitude 
df_merge_col["Latitude"] = latitude 
  
df_merge_col.head() 

print('finished')

In [135]:
search_query = 'apotek'
radius = 10000000
print(search_query + ' .... OK!')

apotek .... OK!


In [136]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=V04KOCNRBTU1RTELZ24JKXDJLSRN4QXCS30BRDK0R2LJMGMH&client_secret=UCCZMNLAWTT2KDJORWDKGNN1UGZ5QWF1OIPOEFWGU3RCXHRH&ll=59.3249529,18.0706429&v=20180604&query=apotek&radius=10000000&limit=1000'

In [137]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2166412321bc05cdc36f53'},
 'response': {'venues': [{'id': '4bac99caf964a52070fd3ae3',
    'name': 'Apotek Hjärtat',
    'location': {'address': 'Krukmakargatan 13',
     'crossStreet': 'Torkel Knutssonsgatan',
     'lat': 59.31753415630597,
     'lng': 18.05790968113217,
     'labeledLatLngs': [{'label': 'display',
       'lat': 59.31753415630597,
       'lng': 18.05790968113217}],
     'distance': 1097,
     'postalCode': '118 51',
     'cc': 'SE',
     'city': 'Stockholm',
     'state': 'Storstockholm',
     'country': 'Sverige',
     'formattedAddress': ['Krukmakargatan 13 (Torkel Knutssonsgatan)',
      '118 51 Stockholm',
      'Sverige']},
    'categories': [{'id': '4bf58dd8d48988d10f951735',
      'name': 'Pharmacy',
      'pluralName': 'Pharmacies',
      'shortName': 'Pharmacy',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pharmacy_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1596024

In [138]:
# assign relevant part of JSON to venues
locations = results['response']['venues']

# tranform venues into a dataframe
apotek = json_normalize(locations)


In [139]:
apotek.shape

(50, 18)

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered